In [ ]:
import sklearn

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

np.random.seed(2022)

In [ ]:
def load_data():
    transactions = pd.read_csv('transactions.csv')
    customers = pd.read_csv('customers.csv')
    geo = pd.read_csv('geo.csv')

df = load_data()

In [ ]:
geo = pd.read_csv('geo.csv')
geo

In [ ]:
transactions = transactions[["OFFER_PRICE", "TEST_SET_ID", "OFFER_TYPE", "OFFER_STATUS"]]

In [ ]:
from sklearn.model_selection import train_test_split

def split_validation_test_train(dataframe):
    validation = dataframe[~pd.isna(dataframe['TEST_SET_ID'])]
    X_val = validation.drop(['OFFER_STATUS', 'TEST_SET_ID'], axis=1)
    y_val = validation['TEST_SET_ID']
    data = dataframe[pd.isna(dataframe['TEST_SET_ID'])].drop('TEST_SET_ID', axis=1)
    data['OFFER_STATUS'] = data['OFFER_STATUS'].apply(lambda x: 1 if x in ['WIN', 'Win', 'WON', 'Won'] else 0)
    X = data.drop('OFFER_STATUS', axis=1)
    y = data['OFFER_STATUS']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2022)
    return X_val, X_train, X_test, y_val, y_train, y_test

X_val, X_train, X_test, y_val, y_train, y_test = split_validation_test_train(transactions)
Exploratory = X_train.copy()

In [ ]:
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

numeric_transformer = Pipeline(steps=[
       ('imputer', SimpleImputer(strategy='mean'))
      ,('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
       ('imputer', SimpleImputer(strategy='constant'))
      ,('encoder', OrdinalEncoder())
])

numeric_features = ['OFFER_PRICE']
categorical_features = ['OFFER_TYPE']
preprocessor = ColumnTransformer(
   transformers=[
    ('numeric', numeric_transformer, numeric_features)
   ,('categorical', categorical_transformer, categorical_features)
])

In [ ]:
transactions

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

class DoubledOfferPrice(BaseEstimator, TransformerMixin):
    def transform(self, X):
        doubled_price =  X.loc[: , 'OFFER_PRICE'] * 2
        return np.c_[X, doubled_price]

doubled_offer_price = DoubledOfferPrice()
res = doubled_offer_price.transform(transactions)

In [ ]:
res